In [1]:
from transformers import pipeline
from typing import Optional


# Add new imports for models
sentiment_analyzer = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
topic_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


# # Initialize lighter models
# sentiment_analyzer = pipeline("sentiment-analysis", 
#                             model="distilbert-base-uncased-finetuned-sst-2-english",
#                             device="cpu")

# topic_classifier = pipeline("zero-shot-classification",
#                           model="facebook/bart-large-mnli",
#                           device="cpu")

# Define topics for classification
TOPICS = [
    "billing",
    "customer service",
    "security",
    "ATM issues",
    "wait times",
    "staff behavior",
    "mobile banking",
    "account management",
    "loans",
    "other"
]

/home/aizeria/Documents/Customer-Reviews-of-Bank-Agencies-in-Morocco/ScpPrjEnv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu
Device set to use cpu


In [2]:
def get_sentiment(text: Optional[str]) -> str:
    """Get sentiment from text"""
    if not text:
        return None
    try:
        result = sentiment_analyzer(text)
        # Convert 5-class sentiment to three classes
        score = float(result[0]['label'].split()[0])
        if score > 3:
            return "positive"
        elif score < 3:
            return "negative"
        else:
            return "neutral"
    except Exception as e:
        print(f"Error in sentiment analysis: {str(e)}")
        return None

def get_topic(text: Optional[str]) -> str:
    """Get topic from text"""
    if not text:
        return None
    try:

        result = topic_classifier(
            text, 
            TOPICS,
            multi_label=False,
            hypothesis_template="This text is about {}."
        )
        # Add confidence threshold
        if result['scores'][0] < 0.2:
            return "other"
        return result['labels'][0]
    except Exception as e:
        print(f"Error in topic classification: {str(e)}")
        return None

In [15]:
test_texts = [
        "Great service and helpful staff",  # Should be positive
        "Average experience, nothing special",  # Should be neutral
        "Terrible waiting times and rude staff",  # Should be negative
        "The ATM was out of service",  # Should be "ATM issues"
        "",  # Should handle empty text
        None
    ]
    
for text in test_texts:
    print(f"\nText: {text}")
    print(f"Sentiment: {get_sentiment(text)}")
    print(f"Topic: {get_topic(text)}")


Text: Great service and helpful staff
Sentiment: positive
Topic: customer service

Text: Average experience, nothing special
Sentiment: neutral
Topic: other

Text: Terrible waiting times and rude staff
Sentiment: negative
Topic: staff behavior

Text: The ATM was out of service
Sentiment: negative
Topic: ATM issues

Text: 
Sentiment: None
Topic: None

Text: None
Sentiment: None
Topic: None


In [16]:
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    pipeline
)
import torch

# Configure for CPU usage
device = "cpu"

# Initialize sentiment analyzer
sentiment_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_analyzer = pipeline(
    "sentiment-analysis", 
    model=sentiment_model, 
    tokenizer=sentiment_tokenizer,
    device=device
)

# Initialize topic classifier
topic_model_name = "facebook/bart-large-mnli"
topic_model = AutoModelForSequenceClassification.from_pretrained(topic_model_name)
topic_tokenizer = AutoTokenizer.from_pretrained(topic_model_name)
topic_classifier = pipeline(
    "zero-shot-classification",
    model=topic_model,
    tokenizer=topic_tokenizer,
    device=device
)


Device set to use cpu
Device set to use cpu


In [17]:

# Test the models
test_texts = [
    "Great service and helpful staff",  # Should be positive
    "Average experience, nothing special",  # Should be neutral
    "Terrible waiting times and rude staff",  # Should be negative
    "The ATM was out of service",  # Should be "ATM issues"
    "",  # Should handle empty text
    None  # Should handle None
]

# Run tests
for text in test_texts:
    print(f"\nText: {text}")
    print(f"Sentiment: {get_sentiment(text)}")
    print(f"Topic: {get_topic(text)}")

# Clear CUDA cache if needed
if torch.cuda.is_available():
    torch.cuda.empty_cache()


Text: Great service and helpful staff
Sentiment: positive
Topic: customer service

Text: Average experience, nothing special
Sentiment: neutral
Topic: other

Text: Terrible waiting times and rude staff
Sentiment: negative
Topic: staff behavior

Text: The ATM was out of service
Sentiment: negative
Topic: ATM issues

Text: 
Sentiment: None
Topic: None

Text: None
Sentiment: None
Topic: None
